In [1]:
# i will follow the same problem statement as in lecture 6

In [2]:
# read data
import pandas as pd

In [3]:
# slight issue with the encoding of the csv file
import chardet

with open('datasets/bag_of_words_dataset/spam.csv', 'rb') as f:
    result = chardet.detect(f.read())
    print(result)

{'encoding': 'Windows-1252', 'confidence': 0.7269493857068697, 'language': ''}


In [4]:
messages = pd.read_csv(
    filepath_or_buffer="datasets/bag_of_words_dataset/spam.csv",
    delimiter=',',
    #     names=["label", "message"],
    encoding='Windows-1252',
)

print(messages.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [5]:
# remove unnamed colums and clean the data
messages = messages.drop(columns=["Unnamed: 2","Unnamed: 3","Unnamed: 4"], axis=1)
messages.rename(columns={"v1": "label", "v2": "message"}, inplace=True)
print(messages)

     label                                            message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [6]:
# import regular expression and perform cleansing
import re

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [8]:
wordnet_lemmatizer = WordNetLemmatizer()

In [9]:
# further clean the data 
# messages -> replace all the special characters -> lemmatization -> store the corpus as a list
corpus_list = []

for i in range(len(messages)):
    ind_review = re.sub(pattern="[^a-zA-Z]", repl=" ", string=messages['message'][i])
#     print(ind_review)
    ind_review = ind_review.lower()
    ind_review = ind_review.split()
    ind_review = [wordnet_lemmatizer.lemmatize(word) for word in ind_review if word not in set(stopwords.words("english"))]
    ind_review = " ".join(ind_review)
    corpus_list.append(ind_review)

# lets look at the final data
print(type(corpus_list))
# print(corpus_list)

<class 'list'>


In [10]:
# we will be using sklearn BOW approach
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
# create BOW with default n_gram range of (1,1)
# there are multiple important options: stopwords, binary, lower_case, N-gram
bow_generator = CountVectorizer(max_features=3000) # max features set the total words in the vocabulary
X_train = bow_generator.fit_transform(corpus_list).toarray()

In [12]:
print(type(X_train))
print(X_train.shape)
print(X_train)

<class 'numpy.ndarray'>
(5572, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
print(bow_generator.vocabulary_)
# NOTICE: 'go': np.int64(930), here 930 refers to the index

{'go': np.int64(930), 'point': np.int64(1907), 'crazy': np.int64(501), 'available': np.int64(142), 'bugis': np.int64(312), 'great': np.int64(954), 'world': np.int64(2940), 'la': np.int64(1271), 'cine': np.int64(416), 'got': np.int64(945), 'wat': np.int64(2858), 'ok': np.int64(1747), 'lar': np.int64(1283), 'joking': np.int64(1206), 'wif': np.int64(2903), 'oni': np.int64(1756), 'free': np.int64(864), 'entry': np.int64(717), 'wkly': np.int64(2926), 'comp': np.int64(456), 'win': np.int64(2907), 'fa': np.int64(765), 'cup': np.int64(514), 'final': np.int64(814), 'tkts': np.int64(2683), 'st': np.int64(2487), 'may': np.int64(1484), 'text': np.int64(2620), 'receive': np.int64(2085), 'question': np.int64(2028), 'std': np.int64(2502), 'txt': np.int64(2743), 'rate': np.int64(2058), 'apply': np.int64(98), 'dun': np.int64(653), 'say': np.int64(2238), 'early': np.int64(660), 'already': np.int64(65), 'nah': np.int64(1643), 'think': np.int64(2643), 'usf': np.int64(2789), 'life': np.int64(1330), 'around

In [14]:
# lets try to change the n_gram range
bow_generator = CountVectorizer(max_features=500, ngram_range=(1,2)) # combination of bigram and unigram
X_train = bow_generator.fit_transform(corpus_list).toarray()
print(X_train.shape)

(5572, 500)


In [15]:
# now, lets look at the vocabulary
print(bow_generator.vocabulary_)
# NOTICE: 'please call': np.int64(322)

{'go': np.int64(155), 'point': np.int64(331), 'great': np.int64(164), 'world': np.int64(483), 'got': np.int64(162), 'wat': np.int64(462), 'ok': np.int64(297), 'lar': np.int64(211), 'wif': np.int64(473), 'free': np.int64(142), 'entry': np.int64(124), 'win': np.int64(475), 'st': np.int64(399), 'may': np.int64(250), 'text': np.int64(413), 'receive': np.int64(350), 'question': np.int64(343), 'txt': np.int64(444), 'rate': np.int64(345), 'apply': np.int64(18), 'dun': np.int64(114), 'say': np.int64(364), 'early': np.int64(116), 'already': np.int64(9), 'think': np.int64(420), 'life': np.int64(225), 'around': np.int64(20), 'though': np.int64(422), 'hey': np.int64(185), 'week': np.int64(466), 'word': np.int64(480), 'back': np.int64(31), 'like': np.int64(226), 'fun': np.int64(148), 'still': np.int64(402), 'xxx': np.int64(490), 'send': np.int64(371), 'even': np.int64(126), 'brother': np.int64(47), 'speak': np.int64(397), 'per': np.int64(312), 'set': np.int64(375), 'friend': np.int64(144), 'network

In [16]:
# one should alos change the hyperparameters like ngram_range or max_features to increase the accuracy!